In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
picher_file_path = '../data/picher_stats_2017.csv'
batter_file_path = '../data/batter_stats_2017.csv'
winner = pd.read_csv(picher_file_path)
starter = pd.read_csv(picher_file_path)
batter = pd.read_csv(batter_file_path)

In [4]:
winner.head() #데이터 구조

,선수명,팀명,승,패,세,홀드,블론,경기,선발,이닝,...,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,연봉(2018),연봉(2017)
0,켈리,SK,16,7,0,0,0,30,30,190.0,...,0.76,0.342,73.7,3.60,6.91,3.69,3.44,6.62,140000,85000
1,소사,LG,11,11,1,0,0,30,29,185.1,...,0.53,0.319,67.1,3.88,6.80,3.52,3.41,6.08,120000,50000
2,양현종,KIA,20,6,0,0,0,31,31,193.1,...,0.79,0.332,72.1,3.44,6.54,3.94,3.82,5.64,230000,150000
3,차우찬,LG,10,7,0,0,0,28,28,175.2,...,1.02,0.298,75.0,3.43,6.11,4.20,4.03,4.63,100000,100000
4,레일리,롯데,13,7,0,0,0,30,30,187.1,...,0.91,0.323,74.1,3.80,6.13,4.36,4.31,4.38,111000,85000


In [8]:
winner.columns #칼럼 확인

Index(['선수명', '팀명', '승', '패', '세', '홀드', '블론', '경기', '선발', '이닝', '삼진/9',
       '볼넷/9', '홈런/9', 'BABIP', 'LOB%', 'ERA', 'RA9-WAR', 'FIP', 'kFIP', 'WAR',
       '연봉(2018)', '연봉(2017)'],
      dtype='object')

In [3]:
pd.options.mode.chained_assignment = None #float 형태 출력

In [5]:
# 피처 각각에 대한 scaling을 수행하는 함수를 정의합니다.
def standard_scaling(df, scale_columns):
    for col in scale_columns:
        series_mean = df[col].mean()
        series_std = df[col].std()
        df[col] = df[col].apply(lambda x: (x-series_mean)/series_std)
    return df

In [6]:
columns=[ '패', '세', '홀드', '블론', '경기', '선발', '이닝', '삼진/9',
       '볼넷/9', '홈런/9', 'BABIP', 'LOB%', 'ERA', 'RA9-WAR', 'FIP', 'kFIP', 'WAR',
       '연봉(2018)', '연봉(2017)']
winner_df= standard_scaling(winner,columns)
#예측 하고싶은 열 제외하고 스케일링 

In [7]:
winner_df=winner_df.rename(columns={'승':'y'}) #예측데이터를 Y로 변환
winner_df

,선수명,팀명,y,패,세,홀드,블론,경기,선발,이닝,...,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,연봉(2018),연봉(2017)
0,켈리,SK,16,1.227145,-0.306452,-0.585705,-0.543592,0.059433,2.452068,2.645175,...,-0.442382,0.016783,0.446615,-0.587056,3.174630,-0.971030,-1.058125,4.503142,3.912893,2.734705
1,소사,LG,11,2.504721,-0.098502,-0.585705,-0.543592,0.059433,2.349505,2.547755,...,-0.668521,-0.241686,-0.122764,-0.519855,3.114968,-1.061888,-1.073265,4.094734,3.266495,1.337303
2,양현종,KIA,20,0.907751,-0.306452,-0.585705,-0.543592,0.111056,2.554632,2.706808,...,-0.412886,-0.095595,0.308584,-0.625456,2.973948,-0.837415,-0.866361,3.761956,6.821679,5.329881
3,차우찬,LG,10,1.227145,-0.306452,-0.585705,-0.543592,-0.043811,2.246942,2.350927,...,-0.186746,-0.477680,0.558765,-0.627856,2.740722,-0.698455,-0.760385,2.998081,2.620098,3.333592
4,레일리,롯데,13,1.227145,-0.306452,-0.585705,-0.543592,0.059433,2.452068,2.587518,...,-0.294900,-0.196735,0.481122,-0.539055,2.751570,-0.612941,-0.619085,2.809003,2.975617,2.734705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,장민재,한화,2,0.588357,-0.306452,-0.585705,0.680497,0.214300,-0.112010,0.104300,...,0.363854,0.162874,-1.002712,0.411349,-1.229537,0.375810,0.475992,-0.859111,-0.382416,-0.335587
148,정용운,KIA,3,-0.369825,-0.306452,-0.585705,-0.543592,-0.198678,0.503369,0.042667,...,-0.147418,-0.871003,-0.269422,-0.030253,-0.361721,0.482702,0.622339,-0.874237,-0.369488,-0.535215
149,노경은,롯데,0,-0.369825,-0.306452,-0.585705,-0.543592,-1.024633,-0.419699,-0.850019,...,1.219251,0.466294,-1.356417,1.347354,-1.023431,1.348527,1.389397,-0.964995,-0.288689,-0.020173
150,김승현,삼성,0,-0.050431,-0.306452,-0.364245,-0.543592,0.627278,-0.624825,-0.273451,...,0.226204,0.005545,0.463869,-0.066253,-0.790205,0.728554,0.713175,-1.033063,-0.482608,-0.543201


In [8]:
team_encoding = pd.get_dummies(winner_df['팀명']) #팀명 열 분해
winner_df = winner_df.drop('팀명', axis=1)
winner_df = winner_df.join(team_encoding)

In [9]:
winner_df

,선수명,y,패,세,홀드,블론,경기,선발,이닝,삼진/9,...,연봉(2017),KIA,KT,LG,NC,SK,두산,롯데,삼성,한화
0,켈리,16,1.227145,-0.306452,-0.585705,-0.543592,0.059433,2.452068,2.645175,0.672099,...,2.734705,0,0,0,0,1,0,0,0,0
1,소사,11,2.504721,-0.098502,-0.585705,-0.543592,0.059433,2.349505,2.547755,0.134531,...,1.337303,0,0,1,0,0,0,0,0,0
2,양현종,20,0.907751,-0.306452,-0.585705,-0.543592,0.111056,2.554632,2.706808,0.109775,...,5.329881,1,0,0,0,0,0,0,0,0
3,차우찬,10,1.227145,-0.306452,-0.585705,-0.543592,-0.043811,2.246942,2.350927,0.350266,...,3.333592,0,0,1,0,0,0,0,0,0
4,레일리,13,1.227145,-0.306452,-0.585705,-0.543592,0.059433,2.452068,2.587518,0.155751,...,2.734705,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,장민재,2,0.588357,-0.306452,-0.585705,0.680497,0.214300,-0.112010,0.104300,-0.968896,...,-0.335587,0,0,0,0,0,0,0,0,1
148,정용운,3,-0.369825,-0.306452,-0.585705,-0.543592,-0.198678,0.503369,0.042667,-0.777918,...,-0.535215,1,0,0,0,0,0,0,0,0
149,노경은,0,-0.369825,-0.306452,-0.585705,-0.543592,-1.024633,-0.419699,-0.850019,-0.756699,...,-0.020173,0,0,0,0,0,0,1,0,0
150,김승현,0,-0.050431,-0.306452,-0.364245,-0.543592,0.627278,-0.624825,-0.273451,0.205264,...,-0.543201,0,0,0,0,0,0,0,1,0


In [21]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [22]:
X = winner_df[winner_df.columns.difference(['선수명', 'y'])]
y = winner_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19) #모델 랜덤 선정

In [23]:
lr = linear_model.LinearRegression() #모델 생성
model = lr.fit(X_train, y_train)

In [28]:
predict_2017_win=lr.predict(X_test)#예측

In [30]:
winner_df['예측승']=pd.Series(predict_2017_win) #예측 데이터 기존 데이터에 추가

In [34]:
winner=pd.read_csv(picher_file_path) #기존 데이터와 예측데이터 합치기
winner=winner[['선수명','승']]

winner_df = winner_df.sort_values(by=['y'], ascending=False)
winner_df.drop(['승'], axis=1, inplace=True, errors='ignore')
winner_df = winner_df.merge(winner, on=['선수명'], how='left')
winner_df = winner_df[['선수명', 'y', '예측승']]
winner_df.columns = ['선수명', '실제승', '예측승']

In [35]:
winner_df.head(10)

,선수명,실제승,예측승
0,양현종,20,0.288926
1,켈리,16,12.902510
2,장원준,14,-0.204215
3,레일리,13,5.394351
4,박종훈,12,-0.382415
5,윤성환,12,0.494545
6,박세웅,12,1.639622
7,소사,11,3.639280
8,유희관,11,6.787237
9,송승준,11,0.003575


In [44]:
start_col=['승', '패', '세', '홀드', '블론', '경기', '이닝', '삼진/9',
       '볼넷/9', '홈런/9', 'BABIP', 'LOB%', 'ERA', 'RA9-WAR', 'FIP', 'kFIP', 'WAR',
       '연봉(2018)', '연봉(2017)']
starter_df= standard_scaling(starter,start_col)

In [45]:
starter_df=starter_df.rename(columns={'선발':'y'})
starter_df

,선수명,팀명,승,패,세,홀드,블론,경기,y,이닝,...,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,연봉(2018),연봉(2017)
0,켈리,SK,3.313623,1.227145,-0.306452,-0.585705,-0.543592,0.059433,30,2.645175,...,-0.442382,0.016783,0.446615,-0.587056,3.174630,-0.971030,-1.058125,4.503142,3.912893,2.734705
1,소사,LG,2.019505,2.504721,-0.098502,-0.585705,-0.543592,0.059433,29,2.547755,...,-0.668521,-0.241686,-0.122764,-0.519855,3.114968,-1.061888,-1.073265,4.094734,3.266495,1.337303
2,양현종,KIA,4.348918,0.907751,-0.306452,-0.585705,-0.543592,0.111056,31,2.706808,...,-0.412886,-0.095595,0.308584,-0.625456,2.973948,-0.837415,-0.866361,3.761956,6.821679,5.329881
3,차우찬,LG,1.760682,1.227145,-0.306452,-0.585705,-0.543592,-0.043811,28,2.350927,...,-0.186746,-0.477680,0.558765,-0.627856,2.740722,-0.698455,-0.760385,2.998081,2.620098,3.333592
4,레일리,롯데,2.537153,1.227145,-0.306452,-0.585705,-0.543592,0.059433,30,2.587518,...,-0.294900,-0.196735,0.481122,-0.539055,2.751570,-0.612941,-0.619085,2.809003,2.975617,2.734705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,장민재,한화,-0.309907,0.588357,-0.306452,-0.585705,0.680497,0.214300,5,0.104300,...,0.363854,0.162874,-1.002712,0.411349,-1.229537,0.375810,0.475992,-0.859111,-0.382416,-0.335587
148,정용운,KIA,-0.051084,-0.369825,-0.306452,-0.585705,-0.543592,-0.198678,11,0.042667,...,-0.147418,-0.871003,-0.269422,-0.030253,-0.361721,0.482702,0.622339,-0.874237,-0.369488,-0.535215
149,노경은,롯데,-0.827554,-0.369825,-0.306452,-0.585705,-0.543592,-1.024633,2,-0.850019,...,1.219251,0.466294,-1.356417,1.347354,-1.023431,1.348527,1.389397,-0.964995,-0.288689,-0.020173
150,김승현,삼성,-0.827554,-0.050431,-0.306452,-0.364245,-0.543592,0.627278,0,-0.273451,...,0.226204,0.005545,0.463869,-0.066253,-0.790205,0.728554,0.713175,-1.033063,-0.482608,-0.543201


In [46]:
team_encoding = pd.get_dummies(starter_df['팀명'])
starter_df = starter_df.drop('팀명', axis=1)
starter_df = starter_df.join(team_encoding)

In [47]:
starter_df[starter_df.columns.difference(['선수명', 'y'])]
y = starter_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [48]:
lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)

In [50]:
predict_2017_start=lr.predict(X_test)

starter_df['예측선발']=pd.Series(predict_2017_start)

start=pd.read_csv(picher_file_path)
start=start[['선수명','선발']]

start_df = starter_df.sort_values(by=['y'], ascending=False)
start_df.drop(['선발'], axis=1, inplace=True, errors='ignore')
start_df = start_df.merge(start, on=['선수명'], how='left')
start_df = start_df[['선수명', 'y', '예측선발']]
start_df.columns = ['선수명', '실제선발', '예측선발']

In [51]:
start_df.head(10)

,선수명,실제선발,예측선발
0,양현종,31,3.552714e-15
1,켈리,30,3.100000e+01
2,레일리,30,3.000000e+00
3,장원준,29,1.000000e+00
4,문승원,29,NaN
5,유희관,29,2.500000e+01
6,팻딘,29,-8.881784e-15
7,소사,29,-9.769963e-15
8,윤성환,28,2.000000e+00
9,박종훈,28,1.154632e-14


In [53]:
batter.dtypes
batter["BABIP"]=pd.to_numeric(batter["BABIP"],errors='coerce')
batter.columns

Index(['선수명', '팀명', '경기', '타석', '타수', '안타', '홈런', '득점', '타점', '볼넷', '삼진', '도루',
       'BABIP', '타율', '출루율', '장타율', 'OPS', 'wOBA', 'WAR', '연봉(2018)',
       '연봉(2017)'],
      dtype='object')

In [55]:
batter_col=['경기', '타석', '타수', '안타', '득점', '타점', '볼넷', '삼진', '도루',
       'BABIP', '타율', '출루율', '장타율', 'OPS', 'wOBA', 'WAR', '연봉(2018)',
       '연봉(2017)']
batter_df=standard_scaling(batter,batter_col)

In [57]:
batter_df= batter_df.rename(columns={'홈런':'y'})
batter_df=batter_df.fillna(0)
batter_df.head(5)

,선수명,팀명,경기,타석,타수,안타,y,득점,타점,볼넷,...,도루,BABIP,타율,출루율,장타율,OPS,wOBA,WAR,연봉(2018),연봉(2017)
0,최정,SK,1.149505,1.434367,1.231734,1.344928,46,1.871933,2.601791,2.595310,...,-0.448042,0.101238,0.647202,1.064628,2.156064,1.732763,1.380264,3.857382,3.107502,3.264136
1,최형우,KIA,1.413314,1.952604,1.716832,2.074446,26,2.174277,2.823399,3.927160,...,-0.615839,0.460774,0.899677,1.267954,1.424382,1.397014,1.268659,3.798140,4.045622,4.234754
2,박건우,두산,1.171489,1.515659,1.537807,2.092684,20,1.939120,1.493746,1.109785,...,2.740086,0.679622,1.132730,1.038107,1.465031,1.318014,1.212856,3.703353,0.512037,0.012569
3,나성범,NC,1.039584,1.607113,1.624432,2.019732,24,2.342246,2.158573,1.468360,...,2.236697,0.859390,0.948230,0.958544,1.478581,1.290364,1.138453,2.873969,0.699661,0.514054
4,손아섭,롯데,1.457283,2.145672,2.074881,2.384491,20,2.678185,1.557063,3.261235,...,3.579067,0.554566,0.831703,1.002746,1.004343,1.033615,0.971045,2.850272,4.045622,1.484672


In [60]:
team_encoding = pd.get_dummies(batter_df['팀명'])
batter_df = batter_df.drop('팀명', axis=1)
batter_df = batter_df.join(team_encoding)

In [61]:
X = batter_df[batter_df.columns.difference(['선수명', 'y'])]
y = batter_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [62]:
lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)

In [63]:
predict_2017_batter=lr.predict(X_test)

batter_df['예측홈런']=pd.Series(predict_2017_batter)

batter = pd.read_csv(batter_file_path)
batter = batter[['선수명', '홈런']]

batter_df = batter_df.sort_values(by=['y'], ascending=False)
batter_df.drop(['홈런'], axis=1, inplace=True, errors='ignore')
batter_df = batter_df.merge(batter, on=['선수명'], how='left')
batter_df = batter_df[['선수명', 'y', '예측홈런']]
batter_df.columns = ['선수명', '실제홈런', '예측홈런']

In [64]:
batter_df.head(10)

,선수명,실제홈런,예측홈런
0,최정,46,0.860801
1,스크럭스,35,2.506034
2,이대호,34,10.622908
3,로맥,31,2.455855
4,러프,31,8.759414
5,한동민,29,-0.859831
6,나지완,27,0.430965
7,버나디나,27,-1.056378
8,최형우,26,4.961316
9,오재일,26,12.027997
